In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import sys
from google.colab import drive
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.autograd import Function
import argparse
import csv
!nvidia-smi
!pip install torchvision==0.5.0
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
drive.mount('/content/gdrive')

Sat Nov 28 14:47:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!unzip 'gdrive/My Drive/colab/hw3/hw3_data.zip'

串流輸出內容已截斷至最後 5000 行。
 extracting: hw3_data/face/train/27052.png  
 extracting: hw3_data/face/train/20371.png  
 extracting: hw3_data/face/train/05427.png  
 extracting: hw3_data/face/train/12436.png  
 extracting: hw3_data/face/train/26009.png  
 extracting: hw3_data/face/train/05878.png  
 extracting: hw3_data/face/train/06647.png  
 extracting: hw3_data/face/train/12449.png  
 extracting: hw3_data/face/train/02334.png  
 extracting: hw3_data/face/train/25609.png  
 extracting: hw3_data/face/train/36448.png  
 extracting: hw3_data/face/train/04737.png  
 extracting: hw3_data/face/train/12791.png  
 extracting: hw3_data/face/train/35307.png  
 extracting: hw3_data/face/train/27623.png  
 extracting: hw3_data/face/train/01046.png  
 extracting: hw3_data/face/train/02319.png  
 extracting: hw3_data/face/train/13551.png  
 extracting: hw3_data/face/train/30549.png  
 extracting: hw3_data/face/train/16949.png  
 extracting: hw3_data/face/train/33463.png  
 extracting: hw3_data/face/train/2

In [3]:
source_train_label = {}
source_test_label = {}
target_train_label = {}
target_test_label = {}
with open('hw3_data/digits/mnistm/train.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            source_train_label[row[0]]=row[1]
with open('hw3_data/digits/mnistm/test.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            source_test_label[row[0]]=row[1]
with open('hw3_data/digits/svhn/train.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            target_train_label[row[0]]=row[1]
with open('hw3_data/digits/svhn/test.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            target_test_label[row[0]]=row[1]

In [4]:
class Allen(Dataset): 
    def __init__(self, root, mode, domain, transform = None):
        self.transform = transform
        self.mode = mode
        self.domain = domain
        self.common_path = root       
        self.filename = sorted(os.listdir(self.common_path))
        self.len = len(self.filename)

    def __getitem__(self, index):
        path = os.path.join(self.common_path + str(self.filename[index]))
        if self.mode == 'train' and self.domain == 'source':
            label = int(source_train_label[self.filename[index]])
        elif self.mode == 'test' and self.domain == 'source':
            label = int(source_test_label[self.filename[index]])
        elif self.mode == 'train' and self.domain == 'target':
            label = int(target_train_label[self.filename[index]])
        else:
            label = int(target_test_label[self.filename[index]])

        im = Image.open(path)          
        if self.transform is not None:
            im = self.transform(im)

        return im, torch.tensor(label,dtype=torch.long)

    def __len__(self):

        return(self.len)

source_trainset = Allen('hw3_data/digits/mnistm/train/','train','source',transform = transforms.Compose([
    transforms.Resize((32,32)), # 隨機將圖片水平翻轉
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
    transforms.Normalize([0.0,],[1.0,])
]))
target_trainset = Allen('hw3_data/digits/svhn/train/','train','target',transform = transforms.Compose([
    transforms.Resize((32,32)), # 隨機將圖片水平翻轉
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
    transforms.Normalize([0.0,],[1.0,])
]))
target_testset = Allen('hw3_data/digits/svhn/test/','test','target',transform = transforms.Compose([
    transforms.Resize((32,32)), # 隨機將圖片水平翻轉
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
    transforms.Normalize([0.0,],[1.0,])
]))

source_train_loader = DataLoader(source_trainset, batch_size = 64, shuffle = True)
target_train_loader = DataLoader(target_trainset, batch_size = 64, shuffle = True)
target_test_loader = DataLoader(target_testset, batch_size = 64)

print('source 大小 : ',len(source_trainset))
print('target 大小 : ',len(target_trainset))
print('test 大小 : ',len(target_testset))

source 大小 :  60000
target 大小 :  73257
test 大小 :  26032


In [5]:
class MSE(nn.Module):
    def __init__(self):
        super(MSE, self).__init__()

    def forward(self, pred, real):
        diffs = torch.add(real, -pred)
        n = torch.numel(diffs.data)
        mse = torch.sum(diffs.pow(2)) / n

        return mse


class SIMSE(nn.Module):

    def __init__(self):
        super(SIMSE, self).__init__()

    def forward(self, pred, real):
        diffs = torch.add(real, - pred)
        n = torch.numel(diffs.data)
        simse = torch.sum(diffs).pow(2) / (n ** 2)

        return simse


class DiffLoss(nn.Module):

    def __init__(self):
        super(DiffLoss, self).__init__()

    def forward(self, input1, input2):

        batch_size = input1.size(0)
        input1 = input1.view(batch_size, -1)
        input2 = input2.view(batch_size, -1)

        input1_l2_norm = torch.norm(input1, p=2, dim=1, keepdim=True).detach()
        input1_l2 = input1.div(input1_l2_norm.expand_as(input1) + 1e-6)

        input2_l2_norm = torch.norm(input2, p=2, dim=1, keepdim=True).detach()
        input2_l2 = input2.div(input2_l2_norm.expand_as(input2) + 1e-6)

        diff_loss = torch.mean((input1_l2.t().mm(input2_l2)).pow(2))

        return diff_loss

class source_encode(nn.Module):
    def __init__(self):
        super(source_encode, self).__init__()
        self.source_encoder_conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  #64*28*28
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, 1, 1),  #128*28*28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, 1, 1), #256*12*12
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 256, 3, 1, 1), #256*10*10
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, 1, 1), #512*8*8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
    def forward(self, x):
        feature = self.source_encoder_conv(x)
        feature = feature.view(-1, 512)
        return feature

class target_encode(nn.Module):
    def __init__(self):
        super(target_encode, self).__init__()
        self.target_encoder_conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  #64*28*28
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, 1, 1),  #128*28*28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, 1, 1), #256*12*12
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 256, 3, 1, 1), #256*10*10
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, 1, 1), #512*8*8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
    def forward(self, x):
        feature = self.target_encoder_conv(x)
        feature = feature.view(-1, 512)
        return feature

class shared_encode(nn.Module):
    def __init__(self):
        super(shared_encode, self).__init__()
        self.shared_encoder_conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  #64*28*28
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, 1, 1),  #128*28*28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, 1, 1), #256*12*12
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 256, 3, 1, 1), #256*10*10
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, 1, 1), #512*8*8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
    def forward(self, x):
        feature = self.shared_encoder_conv(x)
        feature = feature.view(-1, 512)
        return feature

class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        self.shared_encoder_pred_class = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.shared_encoder_pred_class(x)
        return x

class domainclass(nn.Module):
    def __init__(self):
        super(domainclass, self).__init__()
        self.shared_encoder_pred_domain = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )
    def forward(self, x):
        x = self.shared_encoder_pred_domain(x)
        return x

class decoder(nn.Module):
    def __init__(self, code_size=512):
        super(decoder, self).__init__()
        self.layer = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1),
        )
    def forward(self, x):
        x = self.layer(x)
        return x


In [6]:
private_source = source_encode().to(device)
private_target = target_encode().to(device)
shared = shared_encode().to(device)
C = classification().to(device)
D = domainclass().to(device)
decode = decoder().to(device)
lr = 0.0001
opt_p_s = optim.Adam(private_source.parameters(), lr=lr)
opt_p_t = optim.Adam(private_target.parameters(), lr=lr)
opt_s = optim.Adam(shared.parameters(), lr=lr)
opt_class = optim.Adam(C.parameters(), lr=lr)
opt_domain = optim.Adam(D.parameters(), lr=lr)
opt_decode = optim.Adam(decode.parameters(), lr=lr)

class_criterion = nn.CrossEntropyLoss()
domain_criterion = nn.BCEWithLogitsLoss()

loss_recon1 = MSE()
loss_recon2 = SIMSE()
loss_diff = DiffLoss()

print(private_source)
print(private_target)
print(shared)
print(C)
print(D)
print(decode)


source_encode(
  (source_encoder_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_r

In [7]:
n_epoch = 100
len_dataloader = max(len(source_train_loader),len(target_train_loader))
best_acc = 0.53
for epoch in range(n_epoch):  

    private_source.train()
    private_target.train()
    shared.train()
    C.train()
    D.train()
    decode.train()
    for i, ((source_data, source_label), (target_data, _)) in enumerate(zip(source_train_loader, target_train_loader)):
        # p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        # alpha = 2. / (1. + np.exp(-10 * p)) - 1
        source_data = source_data.expand(source_data.data.shape[0], 3, 32, 32)
        target_data = target_data.expand(target_data.data.shape[0], 3, 32, 32)
        source_data = source_data.to(device)
        source_label = source_label.to(device)
        target_data = target_data.to(device)

        # 我們把source data和target data混在一起，否則batch_norm可能會算錯 (兩邊的data的mean/var不太一樣)
        mixed_data = torch.cat([source_data, target_data], dim=0).to(device)
        domain_label = torch.zeros([source_data.shape[0] + target_data.shape[0], 1]).to(device)
        # 設定source data的label為1
        domain_label[:source_data.shape[0]] = 1

        shared_feature = shared(mixed_data)
        domain_logit = D(shared_feature.detach())
        loss = domain_criterion(domain_logit, domain_label)
        loss.backward()
        opt_domain.step()

        #domain reverse
        domain_logit = D(shared_feature)
        err_domain = domain_criterion(domain_logit, domain_label)

        #class err
        pred_class = C(shared_feature[:source_data.shape[0]])
        err_class = class_criterion(pred_class, source_label)

        #recon1,recon2
        private_source_feature = private_source(source_data)
        private_target_feature = private_target(target_data)
        shared_source = shared_feature[:source_data.shape[0]]
        shared_target = shared_feature[source_data.shape[0]:]

        source = torch.cat([shared_source, private_source_feature], dim=1).view(-1,1024,1,1)
        target = torch.cat([shared_target, private_target_feature], dim=1).view(-1,1024,1,1)

        recon_source = decode(source)
        recon_target = decode(target)

        err_sim1 = loss_recon1(recon_source, source_data)
        err_sim2 = loss_recon2(recon_source, source_data)
        err_sim3 = loss_recon1(recon_target, target_data)
        err_sim4 = loss_recon2(recon_target, target_data)

        #shared diff
        mixed_private = torch.cat([private_source_feature, private_target_feature], dim=0)
        err_diff = loss_diff(shared_feature, mixed_private)

        err = err_class + 0.01*err_sim1 + 0.01*err_sim2 + 0.01*err_sim3 + 0.01*err_sim4 + 0.01*err_diff - 0.1 * err_domain
        err.backward()

        opt_p_s.step()
        opt_p_t.step()
        opt_s.step()
        opt_class.step()
        opt_domain.step()
        opt_decode.step()

        opt_p_s.zero_grad()
        opt_p_t.zero_grad()
        opt_s.zero_grad()
        opt_class.zero_grad()
        opt_domain.zero_grad()
        opt_decode.zero_grad()

    shared.eval()
    C.eval()
    with torch.no_grad():
        correct = 0
        count = 0
        for batch_idx, data in enumerate(target_test_loader):
            im, label = data
            im, label = im.to(device), label.to(device)
            im = im.expand(im.data.shape[0], 3, 32, 32)

            result = shared(im)
            result = C(result)

            pred = result.data.max(1, keepdim=True)[1]
            for j in range(len(pred)):
                if pred[j] == label[j]:
                    correct+=1
                count+=1
    print('epoch = ',epoch)
    print('test acc = ',correct/count)
    print('===========================================')
    # temp_acc = correct/count
    # if temp_acc > best_acc:
    #     print('good!!!!!!!!!!!!   the current best acc is : ',temp_acc)
    #     best_acc = temp_acc
    #     torch.save(shared,'gdrive/My Drive/colab/hw3/problem4/M_S/shared')
    #     torch.save(C,'gdrive/My Drive/colab/hw3/problem4/M_S/C')
        

KeyboardInterrupt: ignored